**чатбот на рекуррентных нейросетях (Keras+TensorFlow)**

Евгений Борисов <borisov.e@solarl.ru>


----

## Библиотеки

In [1]:
import numpy as np
import re

import pandas as pd
pd.options.display.max_colwidth = 200  


In [2]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

## Учебные данные

In [ ]:
# http://www.manythings.org/anki/

In [3]:
with open('../data/rus-eng/rus.txt','rt',encoding='utf8') as f: 
    pair = pd.DataFrame([ p.split('\t') for p in f.read().split('\n') if p.strip() ])
    # pair = pd.DataFrame([ p.split('%%') for p in f.read().split('\n') if p.strip() ])

In [4]:
# with open('data/all.txt','rt',encoding='utf8') as f: 
#     pair = pd.DataFrame([ p.split('%%') for p in f.read().split('\n') if p.strip() ])

In [5]:
pair.columns=['Q','A']
pair['Q'] = pair['Q'].str.strip()
pair['A'] = pair['A'].str.strip()

In [6]:
ppr(pair)
pair.sample(9)

записей: 336 666


,Q,A
320212,Tom didn't think that he was going to get caught.,"Том не думал, что его поймают."
264733,Tom told me he'd be glad to help me.,"Том сказал мне, что будет рад мне помочь."
303427,There is one thing that I don't understand.,Я одного не понимаю.
306027,I'd like to get married to someone like you.,Я хотела бы выйти замуж за кого-нибудь вроде Вас.
178604,Free snacks will be provided.,Будут предоставлены бесплатные закуски.
258060,"You're not a veterinarian, are you?",Ты ведь не ветеринар?
127008,I'll be meeting with Tom.,У меня будет встреча с Томом.
136033,Do it as soon as possible.,Сделай это как можно скорее.
172126,This isn't completely wrong.,Это не полностью неверно.


## Чистим тексты

In [7]:
pair['Q_clean'] = pair['Q'].str.lower()
pair['Q_clean'] = pair['Q_clean'].str.replace(r'([,.?!])', r' \1 ')

# pair['Q_clean'] = pair['Q_clean'].apply(lambda s: re.sub( r'\([,.?!]\)', ' \1 ', s))
#pair['Q_clean'] = pair['Q_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
#pair['Q_clean'] = pair['Q_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 

pair['A_clean'] = pair['A'].str.lower()
pair['A_clean'] = pair['A_clean'].str.replace(r'([,.?!])', r' \1 ')

# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'(\W)', ' \1 ', s))
# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 

In [8]:
# добавляем "служебные" слова - начало и конец последовательности
pair['Q_clean'] = pair['Q_clean'].str.split() + ['<START>']
pair['A_clean'] = pair['A_clean'].str.split() + ['<EOS>']

In [9]:
pair[['Q_clean','A_clean']].sample(9)

,Q_clean,A_clean
199907,"[tom, has, never, seen, mary, smile, ., <START>]","[том, никогда, не, видел, ,, как, мэри, улыбается, ., <EOS>]"
195224,"[i, used, to, have, very, long, hair, ., <START>]","[у, меня, раньше, были, очень, длинные, волосы, ., <EOS>]"
261701,"[if, the, alarm, rings, ,, walk, ,, don't, run, ., <START>]","[если, раздастся, сигнал, тревоги, ,, идите, ,, а, не, бегите, ., <EOS>]"
68988,"[you, proved, me, wrong, ., <START>]","[ты, доказал, ,, что, я, неправ, ., <EOS>]"
149400,"[both, of, them, are, very, cute, ., <START>]","[они, оба, очень, симпатичные, ., <EOS>]"
282081,"[i, promise, this, will, never, happen, again, ., <START>]","[обещаю, ,, что, это, больше, не, повторится, ., <EOS>]"
174308,"[tom, thought, nobody, was, home, ., <START>]","[том, думал, ,, что, никого, нет, дома, ., <EOS>]"
258135,"[""where's, your, book, ?, "", ""on, the, table, ., "", <START>]","[""где, ваша, книга, ?, "", -, ""на, столе"", ., <EOS>]"
70185,"[do, you, have, a, family, ?, <START>]","[у, тебя, есть, семья, ?, <EOS>]"


---

In [10]:
# считаем количество слов
pair['lenQ'] = pair['Q_clean'].str.len()
pair['lenA'] = pair['A_clean'].str.len()
pair.describe()

,lenQ,lenA
count,336666.000000,336666.000000
mean,8.022399,7.499317
std,2.307285,2.412566
min,3.000000,3.000000
25%,6.000000,6.000000
50%,8.000000,7.000000
75%,9.000000,9.000000
max,47.000000,50.000000


In [11]:
# определяем максимальную длинну последовательности
pair['lenQ'].quantile(0.95),  pair['lenA'].quantile(0.95)

(12.0, 12.0)

In [12]:
# выкидываем слишком длинные последовательности
n_max = 11
ppr(pair)
pair = pair[ (pair['lenQ']<n_max) & (pair['lenA']<n_max) ]
ppr(pair)

записей: 336 666
записей: 283 850


In [13]:
pair['Q_clean'].str.len().max(), pair['A_clean'].str.len().max()

(10, 10)

In [14]:
# выстраиваем входные последовательности в обратном порядке
# и выравниваем длинну последовательностей,
# дополняем короткие словом "служебным" словом,
pad = ['<PAD>']*n_max
pair['Q_clean'] = pair['Q_clean'].apply( lambda t: pad[len(t):] + list(reversed(t)) )
pair['A_clean'] = pair['A_clean'].apply( lambda t: t + pad[len(t):] )

In [15]:
pair[['Q_clean','A_clean']].sample(9)

,Q_clean,A_clean
20999,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, ., blood, only, it's]","[это, просто, кровь, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
219550,"[<PAD>, <PAD>, <PAD>, <START>, ., fishing, about, much, know, don't, i]","[я, не, очень, разбираюсь, в, рыбалке, ., <EOS>, <PAD>, <PAD>, <PAD>]"
24714,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, ., killed, was, cop, a]","[убили, полицейского, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
224072,"[<PAD>, <PAD>, <PAD>, <START>, ., minutes, ten, in, leaves, train, the]","[поезд, отправляется, через, десять, минут, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>]"
54760,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, ?, eat, tigers, do, what]","[что, едят, тигры, ?, <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
71680,"[<PAD>, <PAD>, <PAD>, <PAD>, <START>, ?, that, like, you, do, how]","[каково, ,, а, ?, <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
45305,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, ?, you, scare, tom, does]","[том, вас, пугает, ?, <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
177993,"[<PAD>, <PAD>, <PAD>, <START>, ., right, is, believe, you, what, do]","[делай, так, ,, как, считаешь, нужным, ., <EOS>, <PAD>, <PAD>, <PAD>]"
172707,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, ., electrocution, from, died, tom]","[том, умер, от, поражения, электрическим, током, ., <EOS>, <PAD>, <PAD>, <PAD>]"


## Кодируем тексты

In [16]:
data = pair['Q_clean'].values.tolist() + pair['A_clean'].values.tolist()

In [17]:
%%time

from gensim.models.word2vec import Word2Vec

w2v_size = 256

w2v = Word2Vec( data, min_count=1, size=w2v_size, window=4, workers=4)


Wall time: 16.5 s


In [18]:
w2v_vocab = sorted([w for w in w2v.wv.vocab])
ppr(w2v_vocab)

записей: 59 935


In [19]:
ii = np.random.permutation(len(w2v_vocab))[:30]
for i in ii:
    w = w2v_vocab[i]
    ww = [ v[0] for v in w2v.wv.most_similar(w, topn=5) ]
    print( w,':',ww )

clothing : ['quality', 'adventure', 'california', 'pneumonia', 'gardening']
заказали : ['пересекли', 'собрали', 'разбили', 'держим', 'учтём']
поджог : ['лев', 'рисковый', 'дикое', 'набожная', 'кудо']
too : ['so', 'very', 'how', 'fatter', 'quite']
александрию : ['каир', 'санитарные', 'антарктиду', 'шашки', 'экспедицию']
обжулили : ['осуждаете', 'осуждаешь', 'объегорили', 'поджидаю', 'подменили']
беспокоило : ['арестуют', 'примеру', 'уволят', 'ограбили', 'доброта']
оратора : ['перебивал', 'бумажные', 'мьянмой', 'жёсткого', 'дешевым']
ладит : ['поздоровался', 'борется', 'гуляет', 'вздохнула', 'коллекционирует']
утрате : ['отце', 'боулинге', 'своём', 'заклад', 'процедуре']
смену : ['сельской', 'расположен', 'спортзал', 'башню', 'пятом']
начос : ['мир', 'паром', 'шаг', 'год', 'ванну']
political : ['colorful', 'стремянка', 'воздыхательница', 'порежу', 'выкопайте']
fantasy : ['chocolates', 'gases', 'pint', 'byron', 'glimpse']
scatters : ['impatiently', 'kojin', 'inwardly', 'обучение', 'golden

In [20]:
pair['Q_code'] = pair['Q_clean'].apply(lambda t: [ w2v.wv.get_vector(w) for w in t ] )
pair['A_code'] = pair['A_clean'].apply(lambda t: [ w2v.wv.get_vector(w) for w in t ] )

In [21]:
pair[['Q_code','A_code']].sample(3)

,Q_code,A_code
1175,"[[-0.7616904, -0.22355564, 0.7845089, 2.3504047, 1.4498441, -1.6410388, -0.78428465, 0.36938807, -0.034999583, -2.723392, -0.31392875, -0.008460425, 0.046154145, 1.661522, -0.52883697, 0.07711175,...","[[-1.364893, -0.71348006, 0.30246612, 1.175325, 0.081364244, -2.2377713, -0.4516037, 0.43250036, -2.0598958, -1.8450876, -0.9010532, -0.5012968, -0.92856264, -0.053666994, 0.014360019, -1.597731, ..."
17966,"[[-0.7616904, -0.22355564, 0.7845089, 2.3504047, 1.4498441, -1.6410388, -0.78428465, 0.36938807, -0.034999583, -2.723392, -0.31392875, -0.008460425, 0.046154145, 1.661522, -0.52883697, 0.07711175,...","[[1.8747137, 0.74694943, 0.80848646, 0.7376907, 0.116412826, 0.5832628, 0.96449316, 1.1086485, 0.73085946, 1.981311, 0.90684104, -0.9181454, 0.5883593, 1.7621347, 0.72139907, -2.4719558, 0.6503424..."
8905,"[[-0.7616904, -0.22355564, 0.7845089, 2.3504047, 1.4498441, -1.6410388, -0.78428465, 0.36938807, -0.034999583, -2.723392, -0.31392875, -0.008460425, 0.046154145, 1.661522, -0.52883697, 0.07711175,...","[[-1.364893, -0.71348006, 0.30246612, 1.175325, 0.081364244, -2.2377713, -0.4516037, 0.43250036, -2.0598958, -1.8450876, -0.9010532, -0.5012968, -0.92856264, -0.053666994, 0.014360019, -1.597731, ..."



-----

In [22]:
ppr(pair)

записей: 283 850


In [23]:
pair = pair.sample(1000)

# pair = pair.sample(283800)
# pair = pair.sample(600)

In [ ]:
x_train = np.stack( pair['Q_code'].values )
y_train = np.stack( pair['A_code'].values )

x_train.shape, y_train.shape

((1000, 11, 256), (1000, 11, 256))

## Строим нейросеть

In [ ]:
features = x_train.shape[2]
timesteps = x_train.shape[1]

# batch_size=100
# num_ex = 1000

In [ ]:
# install git
# pip install git+https://www.github.com/farizrahman4u/recurrentshop.git
# pip install git+https://github.com/farizrahman4u/seq2seq.git

In [ ]:
# import seq2seq
from seq2seq.models import SimpleSeq2Seq

model = SimpleSeq2Seq(input_dim=features, 
                      hidden_dim=features, 
                      output_length=timesteps, 
                      output_dim=features, 
                      depth=2)

Using TensorFlow backend.


In [ ]:
model.compile(loss='mse', optimizer='rmsprop')
# model.compile(loss='mse', optimizer='adam')
# model.compile(loss='mse', optimizer='sgd')


In [ ]:
# model.summary()

In [ ]:
%%time 

history = model.fit(x_train, y_train,
            epochs=500,
            batch_size=500,
            # validation_data=(x_val, y_val)
         )

Epoch 1/500
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5412
Epoch 2/500
1000/1000 [==============================] - 1s 687us/step - loss: 0.4404
Epoch 3/500
1000/1000 [==============================] - 1s 687us/step - loss: 0.3961
Epoch 4/500
1000/1000 [==============================] - 1s 687us/step - loss: 0.3813
Epoch 5/500
1000/1000 [==============================] - 1s 687us/step - loss: 0.3932
Epoch 6/500
1000/1000 [==============================] - 1s 687us/step - loss: 0.3755
Epoch 7/500
1000/1000 [==============================] - 1s 686us/step - loss: 0.3662
Epoch 8/500
1000/1000 [==============================] - 1s 688us/step - loss: 0.3984
Epoch 9/500
1000/1000 [==============================] - 1s 687us/step - loss: 0.3696
Epoch 10/500
1000/1000 [==============================] - 1s 686us/step - loss: 0.3620
Epoch 11/500
1000/1000 [==============================] - 1s 690us/step - loss: 0.3642
Epoch 12/500
1000/1000 [==============================

## Проверяем результат

In [ ]:
i=np.random.randint(len(pair))
x = x_train[i:i+1]

s = [ w2v.wv.similar_by_vector(v)[0][0] for v in np.flipud(x.reshape([timesteps,features]) ) ] 
' '.join( s[0:s.index('<START>')] )

In [ ]:
# y = autoencoder.predict(x)
y = model.predict(x)

s = [ w2v.wv.similar_by_vector(v)[0][0] for v in y.reshape([timesteps,features]) ]
s
#' '.join( s[0:s.index('<EOS>')] )

----

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
# acc = history.history['acc']
#val_acc = history.history['val_acc']
loss = history.history['loss']
#val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# plt.clf()   # clear figure
# acc_values = history_dict['acc']
# val_acc_values = history_dict['val_acc']

# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()

---

In [ ]:
# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None, features))
# # We set up our decoder to return full output sequences,
# # and to return internal states as well. We don't use the
# # return states in the training model, but we will use them in inference.
# decoder_lstm = LSTM(features, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

In [ ]:
# decoder_dense = Dense(features, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# # from tensorflow.keras.layers import Dense

In [ ]:
# схема many2many со сдвигом

# model = Sequential()

# model.add( LSTM(units=features, input_shape=(timesteps, features),   ) ) 
# model.add( LSTM(units=features ) )
# model.add( LSTM(units=features, return_sequences=True, input_shape=(timesteps,features)) )
# model.add( LSTM(units=features, return_sequences=True) )

In [ ]:
# # model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
# # model.compile(loss='mean_squared_error', optimizer='sgd')
# model.compile(loss='mean_squared_error', optimizer='rmsprop')

In [ ]:
# %%time 

# history = model.fit(x_train, y_train,
#           batch_size=100, epochs=50,
#           # validation_data=(x_val, y_val)
#          )

---

In [ ]:
# # from tensorflow.keras.models import Sequential
# from tensorflow.keras import Input
# from tensorflow.keras import Model
# from tensorflow.keras.layers import RepeatVector
# from tensorflow.keras.layers import LSTM


In [ ]:
# inputs = Input((timesteps,features))

# hidden1=hidden2=hidden3=hidden4=features

# #a few many to many layers:
# outputs = LSTM(hidden1,return_sequences=True)(inputs)
# outputs = LSTM(hidden2,return_sequences=True)(outputs)    
# #many to one layer:
# outputs = LSTM(hidden3)(outputs)

# encoder = Model(inputs,outputs)

In [ ]:
# inputs = Input((hidden3,))

# #repeat to make one to many:
# outputs = RepeatVector(timesteps)(inputs)
# #a few many to many layers:
# outputs = LSTM(hidden4,return_sequences=True)(outputs)
# #last layer
# outputs = LSTM(features,return_sequences=True)(outputs)

# decoder = Model(inputs,outputs)

In [ ]:
# inputs = Input((timesteps,features))
# outputs = encoder(inputs)
# outputs = decoder(outputs)
# autoencoder = Model(inputs,outputs)

In [ ]:
# # model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
# # model.compile(loss='mean_squared_error', optimizer='sgd')
# autoencoder.compile(loss='mean_squared_error', optimizer='rmsprop')


In [ ]:
# %%time 

# history = autoencoder.fit(x_train, y_train,batch_size=1, epochs=50 )